### Урок 2. CNN and LSTM for human action recognition#

### -- Автор: Шенк Евгений Станиславович

### Задание 1.
Необходимо нормализовать данные в датасете: написать функцию нормализации и использовать ее внутри класса датасета

### Выполнение:  
    - Созданы 2 класса:
            нормализация mean_std: (x - x.mean()) / x.std()
            нормализация min-max: (x - x.min()) / (x.max() - x.min()
    - На датасете из урока получилось, что нормализация mean_std улучшает результат, а min-max ухудшает, а в Задании 2 нормализация особо на результат не влияла

In [1]:
import torch
from torch import nn
from torch import optim
import torchvision
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
import time
import math
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, Dataset

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

use_cuda = torch.cuda.is_available()
#device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


### Data load

In [2]:
skeletons = pd.read_csv("skels.csv")

In [3]:
LABELS = {0: "cheer up", 1: "jump up", 2:  "hand waving", 3: "sitting down", 4: "clapping"}

In [4]:
skeletons.shape

(636, 3376)

In [5]:
skeletons.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167,3168,3169,3170,3171,3172,3173,3174,3175,3176,3177,3178,3179,3180,3181,3182,3183,3184,3185,3186,3187,3188,3189,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248,3249,3250,3251,3252,3253,3254,3255,3256,3257,3258,3259,3260,3261,3262,3263,3264,3265,3266,3267,3268,3269,3270,3271,3272,3273,3274,3275,3276,3277,3278,3279,3280,3281,3282,3283,3284,3285,3286,3287,3288,3289,3290,3291,3292,3293,3294,3295,3296,3297,3298,3299,3300,3301,3302,3303,3304,3305,3306,3307,3308,3309,3310,3311,3312,3313,3314,3315,3316,3317,3318,3319,3320,3321,3322,3323,3324,3325,3326,3327,3328,3329,3330,3331,3332,3333,3334,3335,3336,3337,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366,3367,3368,3369,3370,3371,3372,3373,3374,labels
0,-0.526048,-0.277147,2.987706,-0.606184,-0.010056,3.010000,-0.681454,0.251614,3.020046,-0.708330,0.386629,3.087164,-0.704835,0.111742,2.856311,-0.515683,-0.024701,2.727949,-0.353615,-0.123308,2.706112,-0.327895,-0.141611,2.749221,-0.511823,0.15791,3.311851,-0.543163,-0.013495,3.108863,-0.496480,-0.181723,2.912678,-0.424105,-0.341968,2.685757,-0.559787,-0.277896,2.867135,-0.187794,-0.440893,2.817494,-0.051320,-0.741197,2.696164,0.035544,-0.772924,2.653029,-0.479626,-0.266888,3.033314,-0.132970,-0.302009,2.766374,-0.026497,-0.672848,3.018918,0.066964,-0.706324,2.934041,-0.663448,0.187231,3.019821,-0.287726,-0.173169,2.772417,-0.374481,-0.201329,2.647500,-0.435660,-0.399829,2.653183,-0.382046,-0.341303,2.709815,-0.522987,-0.302266,2.985698,-0.606354,-0.018129,3.009436,-0.682353,0.250679,3.019942,-0.708845,0.385737,3.086951,-0.703320,0.110657,2.854790,-0.514419,-0.026362,2.726717,-0.351712,-0.124065,2.70648,-0.325711,-0.143484,2.749665,-0.512720,0.156979,3.311748,-0.544056,-0.014422,3.108759,-0.496896,-0.182398,2.912465,-0.422955,-0.339052,2.685333,-0.555938,-0.311383,2.888311,-0.187798,-0.440844,2.817507,-0.051158,-0.741131,2.696049,0.036761,-0.770821,2.654296,-0.475196,-0.299777,3.005829,-0.132759,-0.301510,2.766153,-0.026572,-0.672512,3.018797,0.067243,-0.705909,2.934533,-0.664326,0.185232,3.019666,-0.286222,-0.174110,2.770132,-0.344511,-0.206511,2.714750,-0.432879,-0.389477,2.647004,-0.383055,-0.342364,2.709956,-0.522926,-0.291120,2.983348,-0.606446,-0.017055,3.008816,-0.682771,0.250588,3.019696,-0.709493,0.385637,3.086540,-0.703649,0.110274,2.853598,-0.515603,-0.027185,2.724664,-0.352023,-0.123337,2.722028,-0.318310,-0.146859,2.754510,-0.513136,0.156888,3.311502,-0.544470,-0.014513,3.108513,-0.497013,-0.182307,2.912128,-0.423729,-0.339591,2.684088,-0.557001,-0.279047,2.854529,-0.187803,-0.440799,2.817520,-0.050492,-0.74

In [6]:
# MinMax norm
def normalization_minmax(data):
    data.iloc[:,:-1] = data.iloc[:,:-1].apply((lambda x: (x - x.min()) / (x.max() - x.min())), axis=0)

    return data  

In [7]:
# Normalize a tensor image with mean and standard deviation
def normalization(data):
    data.iloc[:,:-1] = data.iloc[:,:-1].apply((lambda x: (x - x.mean()) / x.std()), axis=0)
    
    return data

In [8]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [9]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path,  transform=None):
        self.data = pd.read_csv(file_path)
        if transform is not None:
            self.transform=transform(self.data)

    def __len__(self):
        if self.transform is not None:
            return len(self.transform)
        else:
            return len(self.data)
    
    def __getitem__(self, idx):
        if self.transform is not None:
            item = np.asarray(self.transform.iloc[idx,:-1]).reshape(45,75)
        else:
            item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
            
        label = self.data.iloc[idx,-1]

        return (item, label) 

In [10]:
dataset = Skeleton_Dataset(file_path = "skels.csv", transform=normalization)

### Train-Test split

In [11]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),int(0.25*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle=True)

### Model

In [12]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
#         self.dr = torch.nn.Dropout2d(0.25)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [13]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)

### Train model

In [14]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)
epoches = 100

In [15]:
all_losses = []
start = time.time()
counter = 0
for epoch in range(epoches):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
    
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            #####сделать более аккуратный вывод для каждой эпохии
            ######сделать обновление графика loss по ходу обучению  
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 1.6263  / cheer up ✗ (jump up)
epoch : 16 iter : 20 (0m 15s) 1.5653  / clapping ✗ (hand waving)
epoch : 33 iter : 10 (0m 30s) 1.5122  / cheer up ✗ (jump up)
epoch : 50 iter : 0 (0m 46s) 1.3180  / sitting down ✗ (jump up)
epoch : 66 iter : 20 (1m 1s) 0.7087  / sitting down ✓
epoch : 83 iter : 10 (1m 18s) 0.7774  / sitting down ✓


### Accuracy

In [16]:
total = 0
right = 0
counter = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)   
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Accuracy of the network:   80.0


### Результат
epoches=300  
Accuracy of the network:   70.0 - без нормализации  
Accuracy of the network:   80.0 - norm mean_std  
Accuracy of the network:   60.0 - norm min_max  

### Задание 2.
a. Сгенерировать меньший датасет из 8-10 классов движения  
b. Изменить функцию разбиения датасета на тестовую и тренировочную часть, увеличив кусок, отводимый на тест  
c. Обучить уже существующую модель (предварительно проанализировав какие параметры нужно менять)  
d. Изменить модель для улучшения качества  
e. Сгенерировать другой датасет с меньшим количеством “кадром” в нашей серии и сравнить с улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению кадров, назвать оптимальное, на ваш взгляд, их количество.  


### Выполнение:  
    - скачан датасет из http://rose1.ntu.edu.sg/datasets/actionrecognition.asp в формате .SKELETON  
    - датасет конвертирован https://github.com/shahroudy/NTURGB-D в .npy  
    - выбраны 10 классов:  
    (LABELS = {0: "cheer up", 1: "jump up", 2:  "hand waving", 3: "sitting down", 4: "clapping",  
                    5:  "tear up paper", 6: "put on glasses", 7: "phone call", 8: "take off jacket", 9: "wipe face"}  
    - датасет получился 9412x3376 весом ~400Mb (Код создания датасета вконце нотбука)
    - Изменено разбиение датасета на 30% Train 70% Test
    - Изменены параметры модели:
            n_hidden = 64
            n_layer = 3
            lstm_dropout = 0.1
            epoches = 100
    - При уменьшении кадров вконце (оставались кадры с 0 и далее) качество сразу падало, При ученьшении кадров вначале (оставались с 16 по 45) качетво не страдало, а скарость обучения увеличевалась. Нормализация положительно на результат не влияла. Поэтому лучший вариант получился: оставить с 16 по 45 кадр, без нормализации.  (Замеры Accuracy находятся далее в нотбуке после модели)

In [17]:
class Skeleton_Dataset(Dataset):
    def __init__(self, file_path, frames=45, transform=None, frame_reduce=None):
        self.frames = frames
        self.data = pd.read_csv(file_path)
        self.transform = transform
        if frame_reduce is not None:
            self.data = frame_reduce(self.data, self.frames)
        if transform is not None:
            self.transform = transform(self.data)

    def __len__(self):
        if self.transform is not None:
            return len(self.transform)
        else:
            return len(self.data)
    
    def __getitem__(self, idx):
        if self.transform is not None:
            item = np.asarray(self.transform.iloc[idx,:-1]).reshape(self.frames,75)
        else:
            item = np.asarray(self.data.iloc[idx,:-1]).reshape(self.frames,75)
            
        label = self.data.iloc[idx,-1]

        return (item, label) 

In [18]:
def change_frame_amount(data, frames=45):
    total_frames = 45
    #data = data.drop(data.columns[-(75*(total_frames-frames)+1):-1], axis = 1) # Удаление вконце
    data = data.drop(data.columns[:(75*(total_frames-frames))], axis = 1) # Удаление вначале

    return data

### Data load

In [19]:
LABELS = {0: "cheer up", 1: "jump up", 2:  "hand waving", 3: "sitting down", 4: "clapping",
                    5:  "tear up paper", 6: "put on glasses", 7: "phone call", 8: "take off jacket", 9: "wipe face"}

In [20]:
skeletons_10 = pd.read_csv("skels_10.csv")

In [21]:
skeletons_10.shape

(9412, 3376)

In [22]:
#skeletons_10.head(2)

In [23]:
#skeletons_10.to_csv('skels_test.csv', index=False)

In [24]:
dataset_10 = Skeleton_Dataset(file_path = "skels_10.csv", frames=45, transform=None, frame_reduce=None)

### Train-Test split

In [25]:
train_part = int(0.30*len(dataset_10))
train_dataset, test_dataset = torch.utils.data.random_split(dataset_10, [train_part, int(len(dataset_10) - train_part)])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle=True)

### Model

In [26]:
class LSTM_net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, layer_num, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, layer_num, dropout=dropout, batch_first=True)
#        self.dr = torch.nn.Dropout2d(0.25)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [27]:
n_hidden = 64
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 3
lstm_dropout = 0.1
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer, lstm_dropout)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 64, num_layers=3, batch_first=True, dropout=0.1)
  (fc): Linear(in_features=64, out_features=10, bias=True)
)

### Train model

In [28]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.Adam(rnn.parameters(),lr=learning_rate)
epoches = 100

In [29]:
all_losses = []
start = time.time()
counter = 0
for epoch in range(epoches):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
    
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            #####сделать более аккуратный вывод для каждой эпохии
            ######сделать обновление графика loss по ходу обучению  
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0

epoch : 0 iter : 0 (0m 0s) 2.3375  / take off jacket ✗ (jump up)
epoch : 2 iter : 146 (0m 19s) 1.8025  / wipe face ✗ (hand waving)
epoch : 5 iter : 115 (0m 40s) 1.1548  / phone call ✗ (wipe face)
epoch : 8 iter : 84 (1m 0s) 1.7280  / take off jacket ✗ (cheer up)
epoch : 11 iter : 53 (1m 20s) 1.5409  / hand waving ✓
epoch : 14 iter : 22 (1m 41s) 1.3158  / take off jacket ✓
epoch : 16 iter : 168 (2m 1s) 0.7233  / cheer up ✓
epoch : 19 iter : 137 (2m 22s) 1.4724  / put on glasses ✗ (take off jacket)
epoch : 22 iter : 106 (2m 43s) 0.9621  / take off jacket ✓
epoch : 25 iter : 75 (3m 3s) 1.3915  / wipe face ✓
epoch : 28 iter : 44 (3m 24s) 0.7623  / hand waving ✓
epoch : 31 iter : 13 (3m 44s) 0.6512  / cheer up ✓
epoch : 33 iter : 159 (4m 4s) 1.3512  / clapping ✓
epoch : 36 iter : 128 (4m 25s) 0.7373  / put on glasses ✓
epoch : 39 iter : 97 (4m 45s) 0.5403  / put on glasses ✓
epoch : 42 iter : 66 (5m 6s) 0.8329  / take off jacket ✓
epoch : 45 iter : 35 (5m 26s) 0.9425  / phone call ✓
epoch :

### Accuracy

In [30]:
total = 0
right = 0
counter = 0
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)   
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Accuracy of the network:   67.47572815533981


### Замеры

n_hidden = 64  
n_joints = 25*3  
n_categories = len(LABELS)  
n_layer = 3  
lstm_dropout = 0.1  
criterion = nn.CrossEntropyLoss()  
learning_rate = 0.0007  
optimizer = optim.Adam(rnn.parameters(),lr=learning_rate)  
epoches = 100  

Все кадры  
<15 min  
45 frames -> Accuracy of the network:   70.63106796116504

Удаление кадров вконце  
<10 min  
30 frames -> Accuracy of the network:   59.70873786407767  
<7 min  
15 frames -> Accuracy of the network:   27.42718446601942  
    
Удаление кадров вначале   
<10 min  
30 frames ->  Accuracy of the network:   70.87378640776699  
<7 min  
15 frames -> Accuracy of the network:   58.737864077669904  

### Create DataSet

In [31]:
"""
frames_limit = 45
labels_codes = {
'A022': 0, 'A027':1, 'A023':2, 'A008':3, 'A010':4,
'A013': 5, 'A018':6, 'A028':7, 'A015':8, 'A037':9,
}
"""

"\nframes_limit = 45\nlabels_codes = {\n'A022': 0, 'A027':1, 'A023':2, 'A008':3, 'A010':4,\n'A013': 5, 'A018':6, 'A028':7, 'A015':8, 'A037':9,\n}\n"

In [38]:
"""
%%time

import glob

skels_10 = pd.DataFrame()

for np_name in glob.glob('../data/raw_npy/*.np[yz]'):
    if np_name[-17:-13] in labels_codes.keys():
        x = np.load(np_name, allow_pickle=True).item()
        frames = x['skel_body0'].shape[0]
        label = x['file_name'][-4:]
        x = x['skel_body0'].reshape(frames, 75)[:frames_limit].flatten()
        x = (pd.DataFrame(x)).T
        x['labels'] = labels_codes[label]
        if skels_10.empty:
            skels_10 = x
        else:
            skels_10 = skels_10.append(x)"""

"\n%%time\n\nimport glob\n\nskels_10 = pd.DataFrame()\n\nfor np_name in glob.glob('../data/raw_npy/*.np[yz]'):\n    if np_name[-17:-13] in labels_codes.keys():\n        x = np.load(np_name, allow_pickle=True).item()\n        frames = x['skel_body0'].shape[0]\n        label = x['file_name'][-4:]\n        x = x['skel_body0'].reshape(frames, 75)[:frames_limit].flatten()\n        x = (pd.DataFrame(x)).T\n        x['labels'] = labels_codes[label]\n        if skels_10.empty:\n            skels_10 = x\n        else:\n            skels_10 = skels_10.append(x)"

In [39]:
# skels_10

In [40]:
# skels_10.dropna(inplace=True)

In [41]:
# skels_10.to_csv('skels_10.csv', index=False)

In [42]:
labels_keys = {
'A001': 'drink water', 'A002': 'eat meal', 'A003': 'brush teeth', 'A004': 'brush hair',
'A005': 'drop', 'A006': 'pick up', 'A007': 'throw', 'A008': 'sit down',
'A009': 'stand up', 'A010': 'clapping', 'A011': 'reading', 'A012': 'writing',
'A013': 'tear up paper', 'A014': 'put on jacket', 'A015': 'take off jacket', 'A016': 'put on a shoe',
'A017': 'take off a shoe', 'A018': 'put on glasses', 'A019': 'take off glasses', 'A020': 'put on a hat/cap',
'A021': 'take off a hat/cap', 'A022': 'cheer up', 'A023': 'hand waving', 'A024': 'kicking something',
'A025': 'reach into pocket', 'A026': 'hopping', 'A027': 'jump up', 'A028': 'phone call',
'A029': 'play with phone/tablet', 'A030': 'type on a keyboard', 'A031': 'point to something', 'A032': 'taking a selfie',
'A033': 'check time (from watch)', 'A034': 'rub two hands', 'A035': 'nod head/bow', 'A036': 'shake head',
'A037': 'wipe face', 'A038': 'salute', 'A039': 'put palms together', 'A040': 'cross hands in front',
'A041': 'sneeze/cough', 'A042': 'staggering', 'A043': 'falling down', 'A044': 'headache',
'A045': 'chest pain', 'A046': 'back pain', 'A047': 'neck pain', 'A048': 'nausea/vomiting', 'A049': 'fan self',
'A050': 'punch/slap', 'A051': 'kicking', 'A052': 'pushing', 'A053': 'pat on back',
'A054': 'point finger', 'A055': 'hugging', 'A056': 'giving object', 'A057': 'touch pocket',
'A058': 'shaking hands', 'A059': 'walking towards', 'A060': 'walking apart'}

In [43]:
labels_codes = {
'A022': 0, 'A027':1, 'A023':2, 'A008':3, 'A010':4,
'A013': 5, 'A018':6, 'A028':7, 'A015':8, 'A037':9,
}